In [1]:
import os
from typing import Annotated
from IPython.display import display, Markdown, Latex

from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.open_ai import AzureAssistantAgent
from semantic_kernel.contents.streaming_file_reference_content import StreamingFileReferenceContent
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import (
    KernelFunctionSelectionStrategy,
)
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import (
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt


from dotenv import load_dotenv

load_dotenv(override=True)


True

In [6]:

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")  
AZURE_OPENAI_API_VERSION = "2024-05-01-preview" 

In [7]:

service_id = "code_gen_agent"
kernel = Kernel()

# add the azure openai chat completion serice to the kernel
kernel.add_service(AzureChatCompletion(
    service_id="code_gen_agent",
    api_key=AZURE_OPENAI_API_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT,
))

code_gen_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name="CodeGeneratorAgent", # spaces not allowed
    # instructions="Repeat the user message in the voice of a pirate and then end with a parrot sound.",
    instructions="""
        You are an agent that can write python code.
        You are here to write code that helps answer questions or solve problems.
        ONLY provide code, NOT explanations or the exepcted output.
    """
)

In [10]:
service_id = "code_runner_agent"

code_runner_agent = await AzureAssistantAgent.create(
    service_id=service_id,
    kernel=kernel,
    name="CodeRunnerAgent",
    api_key=AZURE_OPENAI_API_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT,
    instructions="""
        you are agent that CANNOT write code but can run code.
        YOU DONT HAVE ABILITY TO WRITE CODE, ONLY RUN WHAT WAS PROVIDED.
        If the input does not contain any code, say "NO CODE PROVIDED, FINAL ANSWER" and end the conversation.
        if the code is given, run the given code, and answer with only "output: <output>, FINAL ANSWER"
        MAKE SURE TO SAY "FINAL ANSWER" to end the conversation
        YOU DONT HAVE ABILITY TO WRITE CODE, ONLY REVIEW.
    """,
    enable_code_interpreter=True,
)

print(code_runner_agent.ai_model_id)

gpt-4o-mini


In [9]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    return kernel

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.

    Choose only from these participants:
    - CodeGeneratorAgent
    - CodeRunnerAgent

    Always follow these rules when selecting the next participant:
    - After user input, it is CodeGeneratorAgent's turn.
    - After CodeGeneratorAgent replies with relevent code, it is CodeRunnerAgent's turn.

    History:
    {{{{$history}}}}
    """,
)

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the copy has been approved.  If so, respond with a phrase: FINAL ANSWER.

    History:
    {{$history}}
    """,
)

try:
    chat = AgentGroupChat(
        agents=[code_gen_agent, code_runner_agent],
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion("selection"),
            result_parser=lambda result: str(result.value[0]) if result.value is not None else "CodeRunnerAgent",
            agent_variable_name="agents",
            history_variable_name="history",
        ),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[code_runner_agent],
            function=termination_function,
            kernel=_create_kernel_with_chat_completion("termination"),
            result_parser=lambda result: "FINAL ANSWER" in str(result.value[0]),
            history_variable_name="history",
            maximum_iterations=5,
        )
    )

    input = "what are the values in the Fibonacci sequence that that are less then the value of 101?"

    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
    print(f"# {AuthorRole.USER}: '{input}'")

    async for content in chat.invoke():
        print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {chat.is_complete}")
except Exception as e:
    await chat.reset()

# AuthorRole.USER: 'what are the values in the Fibonacci sequence that that are less then the value of 101?'
# AuthorRole.ASSISTANT - CodeGeneratorAgent: '```python
def fibonacci_less_than(n):
    fib_sequence = []
    a, b = 0, 1
    while a < n:
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

fibonacci_values = fibonacci_less_than(101)
print(fibonacci_values)
```'
# AuthorRole.ASSISTANT - CodeRunnerAgent: 'def fibonacci_less_than(n):
    fib_sequence = []
    a, b = 0, 1
    while a < n:
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

fibonacci_values = fibonacci_less_than(101)
fibonacci_values'
# AuthorRole.ASSISTANT - CodeRunnerAgent: 'output: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89], FINAL ANSWER'
# IS COMPLETE: True
